In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

D:\Anaconda\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [18]:
from data_process import kmeans

D:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


MemoryError: 

In [2]:
train=pd.read_csv('data/train.csv')
#test=pd.read_csv('data/test.csv')

In [6]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_194,var_195,var_196,var_197,var_198,var_199,kmeans_cluster100,kmeans_cluster500,kmeans_cluster100_pct,kmeans_cluster500_pct
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914,14,460,0.097993,0.123077
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518,23,240,0.113450,0.103352
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965,94,129,0.108506,0.085417
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996,18,389,0.088994,0.095355
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104,30,238,0.105107,0.090909


In [7]:
features = [c for c in train.columns if c not in ['ID_code', 'target']]

## Feature engineering

In [4]:
for feature in features:
    train[feature] = np.round(train[feature], 2)

#### for var below, create both r2 and keep raw, for labels make a target encoding

- var_12 replace value <=13.56 with Nan, r2_var_16 in [10.7,10.9], var_37>=11.1 with NaN, r2_var_40 label >=12
- r2_var_41 label if in [8.75, 9.05], var_53 >=7.745 label or replace with Nan
- r2_var_80 in [1.75, 2.6] label, r2_var_81<=9.25 label or Nan
- r2_var_89 >=12.2 label or Nan, r2_var_99>=3.8 label or Nan
- r2_var_108 in [14.1988, 14.2013] label
- r2_var_126 <=11.54 label or NAN
- r2_var_134 >=8.8 label or Nan
- r2_var_153 <=12.5 label or Nan
- r2_var_158 <=1.5 label or Nan
- r2_var_175 in [11.7,11.8]
- r2_var_180 >=9.5 label or Nan
- r2_var_196 >=13.75 label or Nan

var_68 only has 451 unique values (what to do)

In [5]:
edge_list=['var_12','var_37','var_40','var_53','var_81','var_89','var_99','var_126','var_134','var_153','var_158','var_180','var_196']
for var in edge_list:
    train['r2_'+var]=np.round(train[var], 2)

train['nan_var_12']=np.where(train['r2_var_12']<=13.56, np.nan, train['r2_var_12'])
train['nan_var_37']=np.where(train['r2_var_37']>=11.1, np.nan, train['r2_var_37'])
train['nan_var_40']=np.where(train['r2_var_40']>=12, np.nan, train['r2_var_40'])
train['nan_var_53']=np.where(train['r2_var_53']>=7.74, np.nan, train['r2_var_53'])
train['nan_var_81']=np.where(train['r2_var_81']<=9.25, np.nan, train['r2_var_81'])
train['nan_var_89']=np.where(train['r2_var_89']>=12.2, np.nan, train['r2_var_89'])
train['nan_var_99']=np.where(train['r2_var_99']>=3.8, np.nan, train['r2_var_99'])
train['nan_var_126']=np.where(train['r2_var_126']<=11.54, np.nan, train['r2_var_126'])
train['nan_var_134']=np.where(train['r2_var_134']>=8.8, np.nan, train['r2_var_134'])
train['nan_var_153']=np.where(train['r2_var_153']<=12.5, np.nan, train['r2_var_153'])
train['nan_var_158']=np.where(train['r2_var_158']<=1.5, np.nan, train['r2_var_158'])
train['nan_var_180']=np.where(train['r2_var_180']>=9.5, np.nan, train['r2_var_180'])
train['nan_var_196']=np.where(train['r2_var_196']>=13.75, np.nan, train['r2_var_196'])

In [6]:
for var in edge_list:
    test['r2_'+var]=np.round(test[var], 2)

test['nan_var_12']=np.where(test['r2_var_12']<=13.56, np.nan, test['r2_var_12'])
test['nan_var_37']=np.where(test['r2_var_37']>=11.1, np.nan, test['r2_var_37'])
test['nan_var_40']=np.where(test['r2_var_40']>=12, np.nan, test['r2_var_40'])
test['nan_var_53']=np.where(test['r2_var_53']>=7.74, np.nan, test['r2_var_53'])
test['nan_var_81']=np.where(test['r2_var_81']<=9.25, np.nan, test['r2_var_81'])
test['nan_var_89']=np.where(test['r2_var_89']>=12.2, np.nan, test['r2_var_89'])
test['nan_var_99']=np.where(test['r2_var_99']>=3.8, np.nan, test['r2_var_99'])
test['nan_var_126']=np.where(test['r2_var_126']<=11.54, np.nan, test['r2_var_126'])
test['nan_var_134']=np.where(test['r2_var_134']>=8.8, np.nan, test['r2_var_134'])
test['nan_var_153']=np.where(test['r2_var_153']<=12.5, np.nan, test['r2_var_153'])
test['nan_var_158']=np.where(test['r2_var_158']<=1.5, np.nan, test['r2_var_158'])
test['nan_var_180']=np.where(test['r2_var_180']>=9.5, np.nan, test['r2_var_180'])
test['nan_var_196']=np.where(test['r2_var_196']>=13.75, np.nan, test['r2_var_196'])

In [27]:
peak_list=['var_16','var_41','var_80','var_108','var_175']
for var in peak_list:
    train['r2_'+var]=np.round(train[var], 2)
    
train['var_16_peak']=np.where(((train['r2_var_16']<=10.9 )& (train['r2_var_16']>=10.7)),1,0)
train['var_41_peak']=np.where(((train['r2_var_41']<=9.05) & (train['r2_var_41']>=8.75)), 1, 0)
train['var_80_peak']=np.where(((train['r2_var_80']<=2.6) & (train['r2_var_80']>=1.75)), 1, 0)
train['var_108_peak']=np.where(((train['r2_var_108']<=14.2) & (train['r2_var_108']>=14.19)), 1, 0)
train['var_175_peak']=np.where(((train['r2_var_175']<=11.8) & (train['r2_var_175']>=11.7)), 1, 0)

In [28]:
train1=train.query('target==1')

In [29]:
targeting_list=['var_16_peak','var_41_peak','var_80_peak','var_108_peak','var_175_peak']
for target in targeting_list:
    for i in (train[target].unique()):
        n1=sum(train1[target]==i)
        n=sum(train[target]==i)
        train[target+'_pct']= train[target].replace(i, n1/n)

In [7]:
for var in edge_list:
    train=train.drop(var, axis=1)
    train=train.drop('r2_'+var, axis=1)

In [8]:
for var in edge_list:
    test=test.drop(var, axis=1)
    test=test.drop('r2_'+var, axis=1)

### Clustering

In [62]:
quantiles=pd.DataFrame()
for feature in features:
    quantiles[feature+'_quantile'] = pd.qcut(train[feature], 4,labels=[1,2,3,4])

In [74]:
train_new=pd.concat([train, quantiles], axis=1)

In [111]:
y=train['target']
X = train.drop(['target', 'ID_code'], axis=1)

In [79]:
for quantile in quantiles.columns:
    train_new[quantile+'_pct']= train_new[quantile].apply(lambda x: sum(train1[quantile]==x)/sum(train_new[quantile]==x))

In [4]:
train1=train.query('target==1')

In [37]:
train['var_108_peak']=pd.cut(train['var_108'], [13.729,14.1988,14.2013,14.743], labels=[0,1,2])

In [38]:
train['var_108_peak']=train['var_108_peak'].replace(2,0)

In [5]:
for cluster in ['kmeans_cluster100','kmeans_cluster500']:
    print (cluster)
    list1=[]
    list2=[]
    for i in (train[cluster].unique()):
        n1=sum(train1[cluster]==i)
        n=sum(train[cluster]==i)
        list1.append(i)
        list2.append(n1/n)
    #print (list1, list2)    
    train[cluster+'_pct']= train[cluster].replace(list1, list2)

kmeans_cluster100
kmeans_cluster500


In [43]:
for i in (train['var_108_peak'].unique()):
    n1=sum(train1['var_108_peak']==i)
    n=sum(train['var_108_peak']==i)
    train['var_108_peak'+'_pct']= train['var_108_peak'].replace(i, n1/n)

In [273]:
# Count zeros seems not help
train.insert(1, 'zeros', (train[features] == 0).astype(int).sum(axis=1))
test.insert(1, 'zeros', (train[features] == 0).astype(int).sum(axis=1))

In [3]:
kmeans_feats = pd.read_csv('data/kmeans_feat_100.csv')
train = pd.merge(train, kmeans_feats, on='ID_code', how='left')

In [274]:
test = pd.merge(test, kmeans_feats, on='ID_code', how='left')

In [8]:
y=train['target']
X = train.drop(['target', 'ID_code'], axis=1)

In [5]:
ID_code=test['ID_code']
X_test = test.drop(['ID_code'],axis = 1)

### lgb model

In [5]:
def shuffle(x,y,t=4):
    # Shuffle positive samples twice and added to the existing 
    pos_shuffle=[]
    neg_shuffle=[]
    for i in range(t):
        mask=y==1
        x1=x[mask].copy()
        rows=np.arange(x1.shape[0])
        for col in range(x1.shape[1]):
            np.random.shuffle(rows) # shuffle rows in each column
            x1[:,col]=x1[rows][:,col]
        pos_shuffle.append(x1)
    for i in range(t//2):
        mask=y==0
        x1=x[mask].copy()
        rows=np.arange(x1.shape[0])
        for col in range(x1.shape[1]):
            np.random.shuffle(rows) # shuffle rows in each column
            x1[:,col]=x1[rows][:,col]
        neg_shuffle.append(x1)
    pos_shuffle=np.vstack(pos_shuffle)  #stack the dataframe, maybe do contatenate is better
    neg_shuffle=np.vstack(neg_shuffle)
    pos_y = np.ones(pos_shuffle.shape[0]) # make labels for the shuffled results
    neg_y = np.zeros(neg_shuffle.shape[0])
    x = np.vstack([x,pos_shuffle,neg_shuffle]) #augmented data. As pos target is around 0.1, pos_s is 0.2x, neg_s is 0.9x, 2.1x size in total
    y = np.concatenate([y,pos_y,neg_y])
    return x,y

In [6]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [11]:
n_splits = 5 # Number of K-fold Splits
splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True).split(X, y))

In [9]:
import lightgbm as lgb

In [10]:
lgb_param = {
    # This one seems to be better than the lower one
    'bagging_freq': 5,
        'bagging_fraction': 0.4, 
        'boost_from_average':'false',
        'boost': 'gbdt',
        'feature_fraction': 0.05,
        'learning_rate': 0.008,#0.0083
        'max_depth': -1,  
        'metric':'auc',
        'min_data_in_leaf': 80,
        'min_sum_hessian_in_leaf': 10.0,
        'num_leaves': 13, 
        #'num_threads': 8,
        'tree_learner': 'serial',
        'objective': 'binary', 
        'verbosity': 1,
        "boost_from_average": "false",
        #'is_unbalance':True #new
}

In [14]:
lgb_oof = np.zeros(len(X))

feature_importance_df = pd.DataFrame()

In [15]:
for i, (train_idx, valid_idx) in enumerate(splits):  
    print(f'Fold {i + 1}')
    # Data shuffling
    X = np.array(X)
    y = np.array(y)
    X_train = X[train_idx.astype(int)]
    y_train = y[train_idx.astype(int)]
    trn_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X[valid_idx.astype(int)], label=y[valid_idx.astype(int)])
    watchlist = [(trn_data, 'train'), (val_data, 'valid')]
    lgb_clf=lgb.train(lgb_param, trn_data, 500000, valid_sets = [trn_data, val_data], early_stopping_rounds=3000, verbose_eval=1000)
    
    #blend[valid_idx, 0] = lgb_clf.predict(X[valid_idx], num_iteration=lgb_clf.best_iteration)
    lgb_oof[valid_idx] = lgb_clf.predict(X[valid_idx], num_iteration=lgb_clf.best_iteration)
    
    #predictions[:,i*5+t] = lgb_clf.predict(X_test, num_iteration=lgb_clf.best_iteration) 
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = lgb_clf.feature_importance()
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
print("lgb score: {:<8.5f}".format(roc_auc_score(y, lgb_oof)))    

Fold 1
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.899074	valid_1's auc: 0.881065
[2000]	training's auc: 0.909481	valid_1's auc: 0.888579
[3000]	training's auc: 0.917016	valid_1's auc: 0.892642
[4000]	training's auc: 0.92271	valid_1's auc: 0.895381
[5000]	training's auc: 0.927593	valid_1's auc: 0.897293
[6000]	training's auc: 0.931758	valid_1's auc: 0.89829
[7000]	training's auc: 0.935525	valid_1's auc: 0.898995
[8000]	training's auc: 0.938982	valid_1's auc: 0.899213
[9000]	training's auc: 0.942282	valid_1's auc: 0.89943
[10000]	training's auc: 0.945395	valid_1's auc: 0.899488
[11000]	training's auc: 0.948413	valid_1's auc: 0.899534
[12000]	training's auc: 0.951288	valid_1's auc: 0.89949
[13000]	training's auc: 0.954028	valid_1's auc: 0.899519
[14000]	training's auc: 0.956714	valid_1's auc: 0.899546
[15000]	training's auc: 0.959316	valid_1's auc: 0.899475
Early stopping, best iteration is:
[12853]	training's auc: 0.953633	valid_1's auc: 0.89

In [13]:
mean_auc = np.mean(val_aucs)
std_auc = np.std(val_aucs)
all_auc = roc_auc_score(oof['target'], oof['predict'])
print("Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

Mean auc: 0.901333521, std: 0.003069795. All auc: 0.901301168.


In [15]:
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)
predictions.to_csv('lgb_all_predictions.csv', index=None)
sub_df = pd.DataFrame({"ID_code":test["ID_code"].values})
sub_df["target"] = predictions['target']
sub_df.to_csv("submission/lgb_sub_v5.csv", index=False)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
oof = np.zeros(len(X))
#predictions = np.zeros(len(X_test))
feature_importance_df = pd.DataFrame()

for i, (train_idx, valid_idx) in enumerate(splits):  
    print(f'Fold {i + 1}')
    X=np.array(X)
    y=np.array(y)
    x_train = X[train_idx.astype(int)]
    y_train = y[train_idx.astype(int)]
    X_t, y_t = shuffle(x_train, y_train)
    X_t = pd.DataFrame(X_t)
    X_t = X_t.add_prefix('var_')
    
    trn_data = lgb.Dataset(X_t, label=y_t)
    #trn_data = lgb.Dataset(X[train_idx.astype(int)], label=y[train_idx.astype(int)])
    val_data = lgb.Dataset(X[valid_idx.astype(int)], label=y[valid_idx.astype(int)])
    watchlist = [(trn_data, 'train'), (val_data, 'valid')]
                 
    #num_round = 15000
    lgb_clf=lgb.train(lgb_param, trn_data, 500000, valid_sets = [trn_data, val_data], early_stopping_rounds=3000, verbose_eval=1000)
    
    oof[valid_idx] = lgb_clf.predict(X[valid_idx], num_iteration=lgb_clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = lgb_clf.feature_importance()
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    #predictions += lgb_clf.predict(X_test, num_iteration=lgb_clf.best_iteration) /n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(y, oof)))

Fold 1
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.887723	valid_1's auc: 0.877342
[2000]	training's auc: 0.896312	valid_1's auc: 0.884541
[3000]	training's auc: 0.902097	valid_1's auc: 0.889274
[4000]	training's auc: 0.90644	valid_1's auc: 0.892753
[5000]	training's auc: 0.909768	valid_1's auc: 0.895346
[6000]	training's auc: 0.912437	valid_1's auc: 0.897177
[7000]	training's auc: 0.914595	valid_1's auc: 0.898356
[8000]	training's auc: 0.916486	valid_1's auc: 0.899132
[9000]	training's auc: 0.918125	valid_1's auc: 0.899694
[10000]	training's auc: 0.91964	valid_1's auc: 0.900073
[11000]	training's auc: 0.92106	valid_1's auc: 0.900228
[12000]	training's auc: 0.922426	valid_1's auc: 0.900342
[13000]	training's auc: 0.923753	valid_1's auc: 0.900461
[14000]	training's auc: 0.925033	valid_1's auc: 0.900485
[15000]	training's auc: 0.926297	valid_1's auc: 0.900528
[16000]	training's auc: 0.927554	valid_1's auc: 0.900489
[17000]	training's auc: 0.928

In [16]:
pd.set_option('display.max_rows', 300)
feature_importance_df.sort_values('importance',ascending=False)[:300]

,feature,importance,fold
13,var_13,1096,1
108,var_108,1092,1
2,var_2,1054,1
1,var_1,1051,1
133,var_133,1046,1
76,var_76,1044,1
169,var_169,1041,1
12,var_12,1041,1
177,var_177,1038,1
191,var_191,1038,1


In [13]:
feature_importance_df.groupby('feature')['importance'].describe().sort_values(by='mean',ascending=False)

In [30]:
df5.tail(10)

,count,mean,std,min,25%,50%,75%,max
feature,,,,,,,,
var_96,5.0,310.8,72.558252,212.0,257.0,342.0,366.0,377.0
var_41,5.0,307.6,72.782553,202.0,292.0,303.0,340.0,401.0
var_38,5.0,306.2,69.958559,238.0,242.0,310.0,335.0,406.0
var_100,5.0,302.2,76.195144,236.0,265.0,273.0,306.0,431.0
var_124,5.0,294.0,50.709960,237.0,266.0,282.0,316.0,369.0
var_103,5.0,293.8,73.523466,223.0,249.0,283.0,300.0,414.0
var_17,5.0,292.2,76.881077,217.0,230.0,291.0,314.0,409.0
var_27,5.0,257.4,33.418558,220.0,234.0,257.0,270.0,306.0
var_30,5.0,245.2,74.038503,170.0,177.0,239.0,306.0,334.0


In [47]:
l=df5.index.tolist()
l1=l[:100]
#l2=l[1:: 3]
#l3=l[2:: 3]
#l4=l[3:: 5]
#l5=l[4:: 5]
#df5.index

In [48]:
len(l1),len(l2),len(l3)

(100, 67, 66)

In [33]:
df5.to_csv(r'data\feature_importance_lgbm.csv',index=False)

In [17]:
feature_importance_df[feature_importance_df['feature'].str.contains('cluster')].groupby('feature')['importance'].describe()

,count,mean,std,min,25%,50%,75%,max
feature,,,,,,,,
kmeans_cluster100,5.0,301.2,81.223149,188.0,285.0,285.0,339.0,409.0
kmeans_cluster100_pct,5.0,575.0,79.658647,453.0,544.0,592.0,643.0,643.0
kmeans_cluster500,5.0,335.6,85.783448,234.0,290.0,316.0,382.0,456.0
kmeans_cluster500_pct,5.0,734.6,97.884115,593.0,711.0,730.0,779.0,860.0


In [49]:
lgb_param1 = {
    # This one seems to be better than the lower one
    'bagging_freq':5,
        'bagging_fraction': 0.5, 
        'boost_from_average':'false',
        'boost': 'gbdt',
        'feature_fraction': 0.1,
        'learning_rate': 0.01,#0.0083
        'max_depth': -1,  
        'metric':'auc',
        'min_data_in_leaf': 80,
        'min_sum_hessian_in_leaf': 10.0,
        'num_leaves': 13, 
        #'num_threads': 8,
        'tree_learner': 'serial',
        'objective': 'binary', 
        'verbosity': 1,
        "boost_from_average": "false",
        #'is_unbalance':True #new
}

In [50]:
oof = np.zeros(len(X))
#predictions = np.zeros(len(X_test))
#feature_importance_df = pd.DataFrame()

for i, (train_idx, valid_idx) in enumerate(splits):  
    print(f'Fold {i + 1}')
    X1=X[[c for c in X.columns if c in l1]]
    X1=np.array(X1)
    y=np.array(y)
    #x_train = X1[train_idx.astype(int)]
    #y_train = y[train_idx.astype(int)]
    #X_t, y_t = shuffle(x_train, y_train)
    #X_t = pd.DataFrame(X_t)
    #X_t = X_t.add_prefix('var_')
    
    #trn_data = lgb.Dataset(X_t, label=y_t)
    trn_data = lgb.Dataset(X1[train_idx.astype(int)], label=y[train_idx.astype(int)])
    val_data = lgb.Dataset(X1[valid_idx.astype(int)], label=y[valid_idx.astype(int)])
    watchlist = [(trn_data, 'train'), (val_data, 'valid')]
                 
    #num_round = 15000
    lgb_clf=lgb.train(lgb_param1, trn_data, 500000, valid_sets = [trn_data, val_data], early_stopping_rounds=3000, verbose_eval=1000)
    
    oof[valid_idx] = lgb_clf.predict(X1[valid_idx], num_iteration=lgb_clf.best_iteration)
    
    #fold_importance_df = pd.DataFrame()
    #fold_importance_df["feature"] = features
    #fold_importance_df["importance"] = lgb_clf.feature_importance()
    #fold_importance_df["fold"] = i + 1
    #feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    #predictions += lgb_clf.predict(X_test, num_iteration=lgb_clf.best_iteration) /n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(y, oof)))

Fold 1
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.889175	valid_1's auc: 0.870043
[2000]	training's auc: 0.906826	valid_1's auc: 0.882276
[3000]	training's auc: 0.915553	valid_1's auc: 0.886105
[4000]	training's auc: 0.921659	valid_1's auc: 0.887383
[5000]	training's auc: 0.927064	valid_1's auc: 0.887897
[6000]	training's auc: 0.932183	valid_1's auc: 0.887967
[7000]	training's auc: 0.93716	valid_1's auc: 0.887799
[8000]	training's auc: 0.941959	valid_1's auc: 0.887655
Early stopping, best iteration is:
[5875]	training's auc: 0.93156	valid_1's auc: 0.887977
Fold 2
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.889219	valid_1's auc: 0.869089
[2000]	training's auc: 0.906461	valid_1's auc: 0.881553
[3000]	training's auc: 0.915029	valid_1's auc: 0.886096
[4000]	training's auc: 0.92114	valid_1's auc: 0.887615
[5000]	training's auc: 0.926675	valid_1's auc: 0.888119
[6000]	training's auc: 0.931853	valid_1's 

- nan_var_12: 472, r2_var_12: 405, var_12: 456
- nan_var_126: 335, r2_var_126: 272, var_126: 284
- nan_var_134: 448, r2_var_134: 338, var_134: 315
- nan_var_153: 369, r2_var_12: 290, var_12: 294
- nan_var_158: 336, r2_var_158: 4278, var_158: 268
- nan_var_180: 483, r2_var_180: 408, var_180: 413
- nan_var_196: 454, r2_var_196: 375, var_196: 358
- nan_var_37: 399, r2_var_37: 315, var_37: 343
- nan_var_40: 539, r2_var_40: 485, var_40: 405
- nan_var_53: 576, r2_var_53: 480, var_53: 447
- nan_var_81: 544, r2_var_81: 450, var_81: 450
- nan_var_89: 493, r2_var_89: 435, var_89: 406
- nan_var_99: 531, r2_var_99: 451, var_99: 456
- all the new peak and peak_pct has low importance with a maximal of 60

In [ ]:
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
for fold_, (trn_idx, val_idx) in enumerate(splits.split(train.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])
    num_round=1000000
    lgb_clf = lgb.train(lgb_param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3500)
    oof[val_idx] = lgb_clf.predict(train.iloc[val_idx][features], num_iteration=lgb_clf.best_iteration)
    predictions += lgb_clf.predict(test[features], num_iteration=lgb_clf.best_iteration) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(target.values, oof)))

In [22]:
from sklearn.ensemble import ExtraTreesClassifier

D:\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [25]:
oof = np.zeros(len(X))
#predictions = np.zeros(len(X_test))
#feature_importance_df = pd.DataFrame()

for i, (train_idx, valid_idx) in enumerate(splits):  
    print(f'Fold {i + 1}')
    x_train = np.array(X)
    y_train = np.array(y)
    clf = ExtraTreesClassifier(n_estimators=1000, criterion='gini', min_samples_split=400,
                               min_samples_leaf=1, max_features=100, n_jobs=-1)
    clf.fit(x_train[train_idx.astype(int)], y_train[train_idx.astype(int)])
    oof[valid_idx] = clf.predict_proba(x_train[valid_idx])[:,1]

print("CV score: {:<8.5f}".format(roc_auc_score(y, oof)))

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
CV score: 0.82238 


In [30]:
submission=pd.DataFrame()
submission['ID_code']=ID_code
submission['target']=predictions
submission.to_csv('submission\sub_lgb_v7.csv', index=False)